In [3]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db






In [4]:
S_b = 90e3
U_b = 400.0
Z_b = U_b**2/S_b
I_b = S_b/(np.sqrt(3)*U_b)
Omega_b = 2*np.pi*50
R_s = 0.023/Z_b
R_r = 0.024/Z_b
X_s = 0.086/Z_b
X_r = 0.196/Z_b
X_m = 3.7/Z_b

X_0 = X_s + X_m
X1  = X_s + X_r*X_m/(X_r + X_m)
T10 = (X_r + X_m)/(Omega_b*R_r)
X1

0.15307854209445587

In [11]:
S_b = 90e3

U_b = 400.0
Z_b = U_b**2/S_b
I_b = S_b/(np.sqrt(3)*U_b)
Omega_b = 2*np.pi*50
R_s = 0.023/Z_b
R_r = 0.024/Z_b
X_s = 0.086/Z_b
X_r = 0.196/Z_b
X_m = 3.7/Z_b

X_0 = X_s + X_m
X1  = X_s + X_r*X_m/(X_r + X_m)
#X1  = X_s+X_m - X_m**2/(X_r+X_m)
X1

0.15307854209445587

In [6]:
S_b = 90e3
U_b = 400.0
Z_b = U_b**2/S_b
I_b = S_b/(np.sqrt(3)*U_b)
Omega_b = 2*np.pi*50
R_s = 0.023/Z_b
R_r = 0.024/Z_b
X_s = 0.086/Z_b
X_r = 0.196/Z_b
X_m = 3.7/Z_b

X_0 = X_s + X_m
X1  = X_s + X_r*X_m/(X_r + X_m)
X1  = X_s - X_m**2/X_r

T10 = (X_r + X_m)/(Omega_b*R_r)

params = {'S_b':S_b,'U_b':U_b,'I_b':I_b,
          'R_s':R_s,'X_0':X_0,'X1':X1,'T10':T10,   # synnchronous machine d-axis parameters
          'H_m':3.5,'Omega_b':2*np.pi*50,
          'v_0':1,'theta_0':0.0,
          'X_l':0.05}



u_ini_dict = {'P_h':0.1, 'Q_h':0.0}  # for the initialization problem
u_run_dict = {'tau_m':0.8,'Q_c':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['omega_r','e1d','e1q']    # [inductor current, PI integrator]
y_ini_list = ['i_d','i_q','v_h','theta_h','tau_m','Q_c'] # for the initialization problem
y_run_list = ['i_d','i_q','v_h','theta_h','P_h','Q_h'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [14]:
v_d =  -v_h*sin(theta_h) 
v_q =  v_h*cos(theta_h) 

tau_e = e1d*i_d + e1q*i_q

sigma = 1-omega_r

domega_r = 1/(2*H_m)*(tau_m - tau_e)
de1d = Omega_b*sigma*e1q - (e1d + (X_0 - X1)*i_q)/T10
de1q =-Omega_b*sigma*e1d - (e1q - (X_0 - X1)*i_d)/T10

g_1 = R_s*i_d - X1*i_q - v_d + e1d
g_2 = R_s*i_q + X1*i_d - v_q + e1q
g_3 = -P_h - (v_h*v_0*sin(theta_h - theta_0))/X_l
g_4 = -Q_c - Q_h + (v_h*v_0*cos(theta_h - theta_0))/X_l - v_h**2/X_l
g_5 = -P_h - (v_d*i_d + v_q*i_q)
g_6 = -Q_h - (v_q*i_d - v_d*i_q)

h_1 = I_b*(i_d*i_d + i_q*i_q)**0.5

sys = {'name':'imib_milano_3rd',
       'params':params,
       'f':[domega_r,de1d,de1q],
       'g':[g_1,g_2,g_3,g_4,g_5,g_6],
       'g_ini':[g_1,g_2,g_3,g_4,g_5,g_6],
       'x':x_list,
       'y_ini':y_ini_list,
       'y':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h':[h_1]}

sys = db.system(sys)
db.sys2num(sys)

In [15]:
sys['f']

Matrix([
[                                                (-e1d*i_d - e1q*i_q + tau_m)/(2*H_m)],
[e1q*(314.159265358979 - 314.159265358979*omega_r) - (e1d + 1.97654645790554*i_q)/T10],
[e1d*(314.159265358979*omega_r - 314.159265358979) - (e1q - 1.97654645790554*i_d)/T10]])